## Aggregating Data

### Group By
Aggregating data is one of the more common tasks when working with big data.
* How many customers are over 65?
* What is the ratio of men to women?
* Group all emails by their sender.

The function `groupBy()` is one tool that we can use for this purpose.

If you look at the API docs, `groupBy(..)` is described like this:
> Groups the Dataset using the specified columns, so that we can run aggregation on them.

This function is a **wide** transformation - it will produce a shuffle and conclude a stage boundary.

Unlike all of the other transformations we've seen so far, this transformation does not return a `DataFrame`.
* In Scala it returns `RelationalGroupedDataset`
* In Python it returns `GroupedData`

This is because the call `groupBy(..)` is only 1/2 of the transformation.

To see the other half, we need to take a look at it's return type, `RelationalGroupedDataset/GroupedData`.

### RelationalGroupedDataset

If we take a look at the API docs for `RelationalGroupedDataset`, we can see that it supports the following aggregations:

| Method | Description |
|--------|-------------|
| `avg(..)` | Compute the mean value for each numeric columns for each group. |
| `count(..)` | Count the number of rows for each group. |
| `sum(..)` | Compute the sum for each numeric columns for each group. |
| `min(..)` | Compute the min value for each numeric column for each group. |
| `max(..)` | Compute the max value for each numeric columns for each group. |
| `mean(..)` | Compute the average value for each numeric columns for each group. |
| `agg(..)` | Compute aggregates by specifying a series of aggregate columns. |
| `pivot(..)` | Pivots a column of the current DataFrame and perform the specified aggregation. |

With the exception of `pivot(..)`, each of these functions return our new `DataFrame`.

In [0]:
pageviewsDF = (spark.read
  .option("inferSchema", "true") # The default, but not costly w/Parquet
  .parquet("/mnt/training/wikipedia/pageviews/pageviews_by_second.parquet/")
)

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

display(
  pageviewsDF
    .groupBy( col("site") )
    .sum()
)

site,sum(requests)
desktop,8737180972
mobile,4605797962


In [0]:
display(
  pageviewsDF
    .groupBy( col("site") )
    .sum("requests")
)

site,sum(requests)
desktop,8737180972
mobile,4605797962


In [0]:
#with Renamning the column
display(
  pageviewsDF
    .groupBy( col("site") )
    .sum("requests")
    .withColumnRenamed("sum(requests)", "totalRequests")
)

site,totalRequests
desktop,8737180972
mobile,4605797962


In [0]:
display(
  pageviewsDF
    .groupBy( col("site") )
    .count()
)

site,count
desktop,3600000
mobile,3600000


### Using SQL functions

In [0]:
(pageviewsDF
  .filter("site = 'mobile'")
  .select( sum( col("requests")), count(col("requests")), avg(col("requests")), min(col("requests")), max(col("requests")) )
  .show()
)
          
(pageviewsDF
  .filter("site = 'desktop'")
  .select( sum( col("requests")), count(col("requests")), avg(col("requests")), min(col("requests")), max(col("requests")) )
  .show()
)

+-------------+---------------+------------------+-------------+-------------+
sum(requests)|count(requests)| avg(requests)|min(requests)|max(requests)|
+-------------+---------------+------------------+-------------+-------------+
 4605797962| 3600000|1279.3883227777778| 645| 3292|
+-------------+---------------+------------------+-------------+-------------+

+-------------+---------------+------------------+-------------+-------------+
sum(requests)|count(requests)| avg(requests)|min(requests)|max(requests)|
+-------------+---------------+------------------+-------------+-------------+
 8737180972| 3600000|2426.9947144444445| 1312| 5695|
+-------------+---------------+------------------+-------------+-------------+

In [0]:
#with formatting numbers
(pageviewsDF
  .filter("site = 'mobile'")
  .select( 
    format_number(sum(col("requests")), 0).alias("sum"), 
    format_number(count(col("requests")), 0).alias("count"), 
    format_number(avg(col("requests")), 2).alias("avg"), 
    format_number(min(col("requests")), 0).alias("min"), 
    format_number(max(col("requests")), 0).alias("max") 
  )
  .show()
)

(pageviewsDF
  .filter("site = 'desktop'")
  .select( 
    format_number(sum(col("requests")), 0), 
    format_number(count(col("requests")), 0), 
    format_number(avg(col("requests")), 2), 
    format_number(min(col("requests")), 0), 
    format_number(max(col("requests")), 0) 
  )
  .show()
)

+-------------+---------+--------+---+-----+
 sum| count| avg|min| max|
+-------------+---------+--------+---+-----+
4,605,797,962|3,600,000|1,279.39|645|3,292|
+-------------+---------+--------+---+-----+

+-------------------------------+---------------------------------+-------------------------------+-------------------------------+-------------------------------+
format_number(sum(requests), 0)|format_number(count(requests), 0)|format_number(avg(requests), 2)|format_number(min(requests), 0)|format_number(max(requests), 0)|
+-------------------------------+---------------------------------+-------------------------------+-------------------------------+-------------------------------+
 8,737,180,972| 3,600,000| 2,426.99| 1,312| 5,695|
+-------------------------------+---------------------------------+-------------------------------+-------------------------------+-------------------------------+